In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
dataset = pd.read_csv('combined_data.csv')

dataset.head(10)

,Unnamed: 0,text,sentiment
0,0,So there is no way for me to plug it in here i...,0
1,1,Good case Excellent value.,1
2,2,Great for the jawbone.,1
3,3,Tied to charger for conversations lasting more...,0
4,4,The mic is great.,1
5,5,I have to jiggle the plug to get it to line up...,0
6,6,If you have several dozen or several hundred c...,0
7,7,If you are Razr owner...you must have this!,1
8,8,Needless to say I wasted my money.,0
9,9,What a waste of money and time!.,0


In [3]:
sentences = dataset['text'].tolist()
labels = dataset['sentiment'].tolist()
print(sentences[:10])

['So there is no way for me to plug it in here in the US unless I go by a converter.', 'Good case Excellent value.', 'Great for the jawbone.', 'Tied to charger for conversations lasting more than 45 minutes.MAJOR PROBLEMS!!', 'The mic is great.', 'I have to jiggle the plug to get it to line up right to get decent volume.', 'If you have several dozen or several hundred contacts then imagine the fun of sending each of them one by one.', 'If you are Razr owner...you must have this!', 'Needless to say I wasted my money.', 'What a waste of money and time!.']


In [4]:
# Separate out the sentences and labels into training and test sets

training_size = int(len(sentences) * 0.8) #To get 80% percent of data frfom training
print(training_size)

training_sentences = sentences[0:training_size]
testing_sentences = sentences[training_size:] #Rest after training set will be testing data

labels_size = int(len(labels) * 0.8)
print(labels_size)

training_labels = labels[0:labels_size]
testing_labels = labels[labels_size:]

# Make labels into numpy arrays for use with the network later
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

1593
1593


# Tokenize the dataset
Tokenize the dataset, including padding and OOV

In [5]:
vocab_size = 1000
embedding_dim = 16
max_length = 100
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"


from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=vocab_size,oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
training_sequences = tokenizer.texts_to_sequences(training_sentences)
training_padded = pad_sequences(training_sequences,maxlen=max_length, padding=padding_type, 
                       truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length, padding=padding_type, 
                       truncating=trunc_type)

# Review a Sequence
Let's quickly take a look at one of the padded sequences to ensure everything above worked appropriately.

In [6]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(decode_review(training_padded[1]))
print(training_sentences[1])

good case excellent value ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ?
Good case Excellent value.


# Train a Basic Sentiment Model with Embeddings

In [7]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length = max_length))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(units = 6, activation = 'relu'))
model.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy' , metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 16)           16000     
_________________________________________________________________
flatten (Flatten)            (None, 1600)              0         
_________________________________________________________________
dense (Dense)                (None, 6)                 9606      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 7         
Total params: 25,613
Trainable params: 25,613
Non-trainable params: 0
_________________________________________________________________


In [8]:
model.fit(training_padded, training_labels_final, epochs=15, validation_data=(testing_padded, testing_labels_final))

Epoch 1/15
50/50 [==============================] - 1s 6ms/step - loss: 0.6940 - accuracy: 0.5009 - val_loss: 0.6946 - val_accuracy: 0.4110
Epoch 2/15
50/50 [==============================] - 0s 3ms/step - loss: 0.6928 - accuracy: 0.5229 - val_loss: 0.6954 - val_accuracy: 0.4110
Epoch 3/15
50/50 [==============================] - 0s 3ms/step - loss: 0.6926 - accuracy: 0.5229 - val_loss: 0.6962 - val_accuracy: 0.4110
Epoch 4/15
50/50 [==============================] - 0s 3ms/step - loss: 0.6925 - accuracy: 0.5229 - val_loss: 0.6970 - val_accuracy: 0.4110
Epoch 5/15
50/50 [==============================] - 0s 3ms/step - loss: 0.6923 - accuracy: 0.5229 - val_loss: 0.6978 - val_accuracy: 0.4110
Epoch 6/15
50/50 [==============================] - 0s 3ms/step - loss: 0.6923 - accuracy: 0.5229 - val_loss: 0.6982 - val_accuracy: 0.4110
Epoch 7/15
50/50 [==============================] - 0s 2ms/step - loss: 0.6921 - accuracy: 0.5229 - val_loss: 0.6988 - val_accuracy: 0.4110
Epoch 8/15
50/50 [==

# Get files for visualizing the network
The code below will download two files for visualizing how your network "sees" the sentiment related to each word. Head to http://projector.tensorflow.org/ and load these files, then click the "Sphereize" checkbox.

In [9]:
# First get the weights of the embedding layer
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)

(1000, 16)


In [10]:
import io

# Write out the embedding vectors and metadata
out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
    word = reverse_word_index[word_num]
    embeddings = weights[word_num]
    out_m.write(word + "\n")
    out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

In [11]:
# Download the files
try:
    from google.colab import files
except ImportError:
    pass
else:
    files.download('vecs.tsv')
    files.download('meta.tsv')

# Predicting Sentiment in New Reviews
Now that you've trained and visualized your network, take a look below at how we can predict sentiment in new reviews the network has never seen before.

In [13]:
# Use the model to predict a review   
fake_reviews = ['I love this phone', 'I hate spaghetti', 
                'Everything was cold',
                'Everything was hot exactly as I wanted', 
                'Everything was green', 
                'the host seated us immediately',
                'they gave us free chocolate cake', 
                'not sure about the wilted flowers on the table',
                'only works when I stand on tippy toes', 
                'does not work when I stand on my head']

print(fake_reviews) 

# Create the sequences
padding_type='post'
sample_sequences = tokenizer.texts_to_sequences(fake_reviews)
fakes_padded = pad_sequences(sample_sequences, padding=padding_type, maxlen=max_length)           

print('\nHOT OFF THE PRESS! HERE ARE SOME NEWLY MINTED, ABSOLUTELY GENUINE REVIEWS!\n')              

classes = model.predict(fakes_padded)

# The closer the class is to 1, the more positive the review is deemed to be
for x in range(len(fake_reviews)):
    print(fake_reviews[x])
    print(classes[x])
    print('\n')

['I love this phone', 'I hate spaghetti', 'Everything was cold', 'Everything was hot exactly as I wanted', 'Everything was green', 'the host seated us immediately', 'they gave us free chocolate cake', 'not sure about the wilted flowers on the table', 'only works when I stand on tippy toes', 'does not work when I stand on my head']

HOT OFF THE PRESS! HERE ARE SOME NEWLY MINTED, ABSOLUTELY GENUINE REVIEWS!

I love this phone
[0.60881966]


I hate spaghetti
[0.04142052]


Everything was cold
[0.19194427]


Everything was hot exactly as I wanted
[0.53879076]


Everything was green
[0.40755448]


the host seated us immediately
[0.60881966]


they gave us free chocolate cake
[0.60881966]


not sure about the wilted flowers on the table
[0.01350993]


only works when I stand on tippy toes
[0.60881966]


does not work when I stand on my head
[0.00103199]


